In [14]:
# Uncomment the lines below to save via local variables
#CURDATAPOINT = 1;
#save_str = "TestingHere_";

"TestingHere_"

In [ ]:
# Running multiple simulations with the same settings below (do this from console, DO NOT UNCOMMENT THESE):
#CURDATAPOINT = 0
#save_str = "testing_fits_"
#for j11 = 1:100
#       CURDATAPOINT = j11
#       nbinclude("run_Simulations.ipynb")
#       println(CURDATAPOINT) # Shows progress
# 
#end

In [1]:
include("../BK_functions/bk_setup_script.jl"); 

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Any}, ::Tuple{Int64}) at .\deprecated.jl:57
 [3] include_from_node1(::String) at .\loading.jl:576
 [4] include(::String) at .\sysimg.jl:14
 [5] include_string(::String, ::String) at .\loading.jl:522
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Gianna\.julia\v0.6\IJulia\src\execute_request.jl:158
 [7] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\Gianna\.julia\v0.6\Compat\src\Compat.jl:378
 [8] eventloop(::ZMQ.Socket) at C:\Users\Gianna\.julia\v0.6\IJulia\src\eventloop.jl:8
 [9] (::IJulia.##14#17)() at .\task.jl:335
while loading C:\Users\Gianna\Documents\MBAM-RESP\BK_functions\bk_setup_script.jl, in expression starting on line 11


In [2]:
phi_base = phi0; # Use the base parameters as true parameters to be estimated via fits through noise

In [3]:
metadata = Dict("data_noise"=>"10%", "num_inits"=>24, "init_param_noise"=>"90%", "phi_base"=>phi_base, "phi1_orig"=>phi_base) # phi1_orig key added for historical compatibility 

Dict{String,Any} with 5 entries:
  "phi_base"         => [0.504, 0.394, 0.185, 0.5, 0.00613]
  "data_noise"       => "10%"
  "phi1_orig"        => [0.504, 0.394, 0.185, 0.5, 0.00613]
  "init_param_noise" => "90%"
  "num_inits"        => 24

In [4]:
# Simulate appropriate noisy data from the phi_base parameters via the full model

ydata_noiseless = BK_simulator(phi_base, hcat(x_grid...), model_id=51);
ydata_orig = ydata_noiseless.*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative)
ydata_log = log10(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative, but in log space assay)
;

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] exp(::Array{Float64,1}) at .\deprecated.jl:57
 [3] #BK_simulator#20(::Int64, ::Function, ::Array{Float64,1}, ::Array{Float64,2}) at C:\Users\Gianna\Documents\MBAM-RESP\BK_functions\BK_functions.ipynb:In[1]:30
 [4] (::#kw##BK_simulator)(::Array{Any,1}, ::#BK_simulator, ::Array{Float64,1}, ::Array{Float64,2}) at .\<missing>:0
 [5] include_string(::String, ::String) at .\loading.jl:522
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Gianna\.julia\v0.6\IJulia\src\execute_request.jl:158
 [7] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\Gianna\.julia\v0.6\Compat\src\Compat.jl:378
 [8] eventloop(::ZMQ.Socket) at C:\Users\Gianna\.julia\v0.6\IJulia\src\eventloop.jl:8
 [9] (::IJulia.##14#17)() at .\task.jl:335
while loading In[4], in expression starting on line 3
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] log10(::Array{Float64,

In [7]:
# Choose models, initial conditions and data to fit

model_nums = [51];
init_conds = Array(Any,length(model_nums))
data_sets = Array(Any,length(model_nums))

n = 0;
for model in model_nums
    n += 1
    if model > 13
        init_conds[n] = phi_base; # Appropriate parameter vector for the model
        data_sets[n] = ydata_orig;
    end
end


Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Any}, ::Int64) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Gianna\.julia\v0.6\IJulia\src\execute_request.jl:158
 [5] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\Gianna\.julia\v0.6\Compat\src\Compat.jl:378
 [6] eventloop(::ZMQ.Socket) at C:\Users\Gianna\.julia\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##14#17)() at .\task.jl:335
while loading In[7], in expression starting on line 4
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Any}, ::Int64) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Gianna\.julia\v0.6\IJulia\src\execute_request.jl:158
 [5] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.So

In [10]:
# Generate the "noisy_fits" dictionary where we save the fit results to
noisy_fits = Dict();

In [11]:
# Iterate over all models to be fit and do the fit
using LsqFit
n = 0;
for model in model_nums
    n+=1;
    ydata_tofit = data_sets[n];
    phi_init_base = init_conds[n];

    # Store multiple fits (24 different initial point for LM, takes around 5 sec / fit)
    all_fits = Array(Any, 24)
    all_costs = Array(Any, length(all_fits))

    for i1 = 1:length(all_fits)
        if i1==1
            phi_init = phi_init_base; # Start the first fit from initial params
        else
            phi_init = phi_init_base.*(1+(2*rand(size(phi_init_base))-1)*0.9) # Add up to 90% noise to initial params
        end
        all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model), hcat(x_grid...), ydata_tofit, log(phi_init));
        all_costs[i1] = sum(all_fits[i1].resid.^2);
    end

    # Find the best fit
    phi1 = exp(all_fits[findmin(all_costs)[2]].param);

    # Saving fit results:
    res = Dict()
    res["ydata_tofit"] = ydata_tofit
    res["phi_init_base"] = phi_init_base
    res["phi1"] = phi1
    # Added for historical compatibility
    res["ydata_orig"] = ydata_tofit
    res["phi0_orig"] = phi_init_base
    res["phi1_orig"] = phi1
    res["ydata_log"] = ydata_tofit
    res["phi0_log"] = phi_init_base
    res["phi1_log"] = phi1
    
    
    noisy_fits[model] = res
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Any}, ::Int64) at .\deprecated.jl:57
 [3] macro expansion at .\In[11]:10 [inlined]
 [4] anonymous at .\<missing>:?
 [5] include_string(::String, ::String) at .\loading.jl:522
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Gianna\.julia\v0.6\IJulia\src\execute_request.jl:158
 [7] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\Gianna\.julia\v0.6\Compat\src\Compat.jl:378
 [8] eventloop(::ZMQ.Socket) at C:\Users\Gianna\.julia\v0.6\IJulia\src\eventloop.jl:8
 [9] (::IJulia.##14#17)() at .\task.jl:335
while loading In[11], in expression starting on line 4
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Any}, ::Int64) at .\deprecated.jl:57
 [3] macro expansion at .\In[11]:11 [inlined]
 [4] anonymous at .\<missing>:?
 [5] include_string(::String, ::String) at .\loading.jl:522
 [6] execute_request(::ZMQ.Sock

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] exp(::Array{Float64,1}) at .\deprecated.jl:57
 [3] finite_difference_jacobian!(::LsqFit.#f#5{##37#39,Array{Float64,2},Array{Float64,1}}, ::Array{Float64,1}, ::Array{Float64,1}, ::Array{Float64,2}, ::Symbol) at C:\Users\Gianna\.julia\v0.6\Calculus\src\finite_difference.jl:183
 [4] finite_difference_jacobian at C:\Users\Gianna\.julia\v0.6\Calculus\src\finite_difference.jl:203 [inlined]
 [5] g at C:\Users\Gianna\.julia\v0.6\Calculus\src\derivative.jl:40 [inlined]
 [6] #levenberg_marquardt#1(::Float64, ::Float64, ::Int64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Bool, ::Array{Float64,1}, ::Array{Float64,1}, ::LsqFit.#levenberg_marquardt, ::LsqFit.#f#5{##37#39,Array{Float64,2},Array{Float64,1}}, ::Calculus.#g#5{LsqFit.#f#5{##37#39,Array{Float64,2},Array{Float64,1}},Symbol}, ::Array{Float64,1}) at C:\Users\Gianna\.julia\v0.6\LsqFit\src\levenberg_marquardt.jl:82
 [7] #lmfit#2(::Array{Any,1}, ::Function, ::

In [15]:
save("results/noisy_fits_"*save_str*string(CURDATAPOINT)*".jld", "noisy_fits", noisy_fits, "metadata", metadata);